In [1]:
import pandas as pd
train_df = pd.read_csv("train.csv")
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk


vectorizer = TfidfVectorizer(max_features=6000, tokenizer=tknzr.tokenize)  
# vectorizer = TfidfVectorizer(max_features=6000, tokenizer=nltk.word_tokenize)  
vectorizer.fit(train_df['text'])
X_train = train_df['text']
X_train = vectorizer.transform(X_train)
X_train = X_train.todense()
#X_answer = vectorizer.transform(x_answer)
#X_answer = X_answer.todense()

In [2]:
y_train = train_df['Category']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(list(y_train.unique()))
y_train = le.transform(y_train)

In [3]:
y_train

array([0, 1, 1, ..., 1, 0, 1])

In [4]:
import keras

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train_enc = label_encode(label_encoder, y_train)
# I/O check
input_shape = X_train.shape[1]

output_shape = len(label_encoder.classes_)

from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 6000
X = model_input

# 1st hidden layer
X_W1 = Dense(units=2048)(X)  #2048
H1 = ReLU()(X_W1)

X_Wz = Dense(units=1024)(H1)  # 1024
Hz = ReLU()(X_Wz)

H1_Wa = Dense(units=512)(Hz)  # 512
Ha = ReLU()(H1_Wa)

H1_Wb = Dense(units=256)(Ha)  # 256
Hb = ReLU()(H1_Wb)

H1_Wc = Dense(units=128)(Hb)  # 128
Hc = ReLU()(H1_Wc)

# 2nd hidden layer
H1_W2 = Dense(units=64)(Hc)  # 64
H2 = ReLU()(H1_W2)

X_Wx = Dense(units=32)(H2)  # 32
Hx = ReLU()(X_Wx)

X_Wy = Dense(units=16)(Hx)  # 16
Hy = ReLU()(X_Wy)


# output layer
H2_W3 = Dense(units=output_shape)(Hy)  # 8
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# training setting
epochs = 1
batch_size = 32

from sklearn.model_selection import train_test_split
a,b,c,d = train_test_split(X_train,y_train_enc,test_size=0.05)



In [5]:
# training!
history = model.fit(a,c, 
                    epochs=epochs, 
                    batch_size=batch_size,
                   validation_data=(b,d))

2375/2375 [==============================] - 163s 69ms/step - loss: 0.2739 - accuracy: 0.8838 - val_loss: 0.2425 - val_accuracy: 0.8980


In [5]:
import numpy as np
test = pd.read_csv("test.csv")
X_test = vectorizer.transform(test['text'])
X_test = X_test.todense()

pred_result = model.predict(X_test, batch_size=128)

pred_result = label_decode(label_encoder, pred_result)

print(pred_result[:5])

[1 0 1 0 0]


In [6]:
test['Category'] = pred_result

In [7]:
test = test.drop(['text'], axis=1)
test.head()

,Id,Category
0,60ffed24da8262dd38c916dd517c617641710ede348320...,1
1,cb931d414e634344c782a128d542188d6ec1d325e82dd1...,0
2,f7febf0abfefcb927ee16440eaca62d795392de2d685a6...,1
3,709399d1be2207b8f06dfc9ddb3a0709d9894c02849b88...,0
4,0d1969c03277dfaef7ccfd40f0710bcff4ecd978faa935...,0


In [8]:
test.to_csv('submission.csv',index=0)